# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of a *C++* library

First, import **AutoWIG**.

In [1]:
import autowig

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [2]:
!conda build ../../bin/conda/libbasic -c statiskit -c conda-forge
!conda install -y libbasic --use-local -c statiskit -c conda-forge

BUILD START: libbasic-1.0.0-0
    (actual version deferred until further download or env creation)
updating index in: /home/main/miniconda/conda-bld/linux-64
updating index in: /home/main/miniconda/conda-bld/noarch

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.7.27.1-0      conda-forge
    certifi:         2017.7.27.1-py36_0 conda-forge
    libdev:          1.0.0-py36_0       statiskit  
    ncurses:         5.9-10             conda-forge
    openssl:         1.0.2l-0           conda-forge
    path.py:         10.3.1-py36_0      conda-forge
    pip:             9.0.1-py36_0       conda-forge
    python:          3.6.2-0            conda-forge
    python-scons:    3.0.0-py36_0       statiskit  
    pyyaml:          3.12-py36_1        conda-forge
    readline:        6.2-0              conda-forge
    setuptools:      36.3.0-py36_0      conda-forge
    six:             1.10.0-py36_1      conda-forge
    sqlite:          3.13.0-1           conda-forge
    tk:  

Once the headers have been installed in the system, we parse them with relevant compilation flags.

In [3]:
%%time
import sys
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

CPU times: user 136 ms, sys: 12 ms, total: 148 ms
Wall time: 222 ms


Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [4]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

CPU times: user 56 ms, sys: 8 ms, total: 64 ms
Wall time: 61.7 ms


In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [5]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = '../../src/py/_basic.cpp',
                             decorator = '../../src/py/basic/_basic.py',
                             prefix = 'wrapper_')

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 47.8 ms


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [6]:
%%time
wrappers.write()

CPU times: user 396 ms, sys: 396 ms, total: 792 ms
Wall time: 2.17 s


Here is the list of the generated wrappers (untracked files).

In [7]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../../src/py/_basic.cpp
	../../src/py/_basic.h
	../../src/py/basic/_basic.py
	../../src/py/wrapper_0f744e8d056f5d469a887c7c78eaf8fe.cpp
	../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
	../../src/py/wrapper_92d464d1243554009adc48a065a5be3e.cpp
	../../src/py/wrapper_f926cb231a7f5da09f313cd361ff94c7.cpp

nothing added to commit but untracked files present (use "git add" to track)


And here, we present the wrappers for the `BinomialDistribution` class.

In [8]:
!pygmentize ../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

#include "_basic.h"



namespace autowig
{

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::BinomialDistribution const volatile * get_pointer<class ::BinomialDistribution const volatile >(class ::BinomialDistribution const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_4046a8421fe9587c9dfbc97778162c7d()
{

    double  (::BinomialDistribution::*method_pointer_3a3ff64f25e358a6a10b1cd3b3425b82)(unsigned int const) const = &::BinomialDistribution::pmf;
    double  (::BinomialDistribution::*method_pointer_3c97a500c9575c259d5cbdd76120ff4f)() const = &::BinomialDistribution::get_pi;
    void  (::BinomialDistribution::*method_pointer_d364a0529e33516f8ecbb7dcedd60aa0)(double const) = &::BinomialDistribution::set_pi;
    boost::python::class_< class ::BinomialDistribution, autowig::Held< class ::BinomialDistribution >::Type > class_4046a8421fe9587c9dfbc97778162c7d("BinomialDistribution", "", boost::python::no_init);
    class_4046a842

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [9]:
!conda build ../../bin/conda/python-basic -c statiskit -c conda-forge
!conda install -y python-basic --use-local -c statiskit --force

BUILD START: python-basic-1.0.0-py27_0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.7.27.1-0      conda-forge
    certifi:         2017.7.27.1-py27_0 conda-forge
    coverage:        4.4.1-py27_0       conda-forge
    icu:             58.1-1             conda-forge
    libbasic:        1.0.0-0            local      
    libboost:        1.61.0-py27_0      statiskit  
    libdev:          1.0.0-py27_0       statiskit  
    librun:          1.0.0-0            statiskit  
    ncurses:         5.9-10             conda-forge
    nose:            1.3.7-py27_2       conda-forge
    openssl:         1.0.2l-0           conda-forge
    path.py:         10.3.1-py27_0      conda-forge
    pip:             9.0.1-py27_0       conda-forge
    python:          2.7.13-1           conda-forge
    python-dev:      1.0.0-py27_0       statiskit  
    python-scons:    3.0.0-py27_0       statiskit  
    pyyaml: 

In file included from /home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/converter/arg_to_python.hpp:10:0,
                 from /home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/call.hpp:15,
                 from /home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/object_core.hpp:14,
                 from /home/main/min

In file included from /home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python.hpp:41:0,
                 from build/py/_basic.h:4:
/home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla

gcc -o /home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/lib/python2.7/site-packages/basic/__basic.so @/home/main/miniconda/conda-bld/python-basic_1505288458048/work/build/py/response_file.rsp -shared -L/home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/lib -lboost_python -lpython2.7 -lbasic
scons: done building targets.
INFO:conda_build.build:Packaging python-basic-1.0.0-py27_0
fatal: No names found, cannot describe anything.
compiling .pyc files...
number of files: 5
Fixing permissions
patchelf: file: /home/main/miniconda/conda-bld/python-basic_1505288458048/_b_env_placehold_place

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [10]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [11]:
binomial.pmf(0)

0.5

In [12]:
binomial.pmf(1)

0.5

In [13]:
binomial.n = 0
binomial

In [14]:
binomial.pmf(0)

1.0

In [15]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print error
else:
    raise Exception('A `basic.ProbabilityError` should have been raised')

a probability must be in the interval [0,1]


Here is a report concerning objects wrapped using this notebook.

In [16]:
import fp17
fp17.report(asg)

Headers: 3 (290 SLOC)
Fields: 2 (50.0%)
Methods: 16 (62.5%)
Classes: 4 (100.0%)
